## RCDS Optimization

In this example we demonstrate RCDS optimization.

In [ ]:
# If you encounter the "Initializing libomp.dylib, but found libomp.dylib already initialized." error
# Please run this cell
import os

In [ ]:
import numpy as np
from xopt.generators.sequential import RCDSGenerator
from xopt.vocs import VOCS
from xopt.evaluator import Evaluator
from xopt import Xopt
from tqdm.auto import tqdm
import warnings


warnings.filterwarnings("ignore")

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [ ]:
# Reload packages automatically
%load_ext autoreload
%autoreload 2

### RCDS test problem

This test problem is a 2-D quadratic function.

In [ ]:
def f_test(input_dict):
    p = []
    for i in range(2):
        p.append(input_dict[f"p{i}"])

    obj = np.linalg.norm(p)
    outcome_dict = {"f": obj}

    return outcome_dict

### Run RCDS on the test problem (YAML method)

In [ ]:
YAML = """
stopping_condition:
    name: MaxEvaluationsCondition
    max_evaluations: 100
generator:
    name: rcds
    x0: null
    init_mat: null
    noise: 0.00001
    step: 0.01
evaluator:
    function: __main__.f_test
vocs:
    variables:
        p0: [0, 1]
        p1: [0, 1]
    objectives:
        f: MINIMIZE
"""

X = Xopt.from_yaml(YAML)
X

In [ ]:
X.random_evaluate(1)
X.run()

Now you can go directly to the [Visualization](#Visualization) section and check out the results.

### Run RCDS on the maximization test problem (YAML method)

In [ ]:
YAML = """
stopping_condition:
    name: MaxEvaluationsCondition
    max_evaluations: 400
generator:
    name: rcds
    x0: null
    init_mat: null
    noise: 1e-8
    step: 0.01
evaluator:
    function: __main__.f_test
vocs:
    variables:
        p0: [1, 3]
        p1: [-4, 2]
    objectives:
        f: MAXIMIZE
"""

X = Xopt.from_yaml(YAML)
X

In [ ]:
X.random_evaluate(1)
X.run()

Now you can go directly to the [Visualization](#Visualization) section and check out the results.

### Run RCDS on the test problem (API method)

#### VOCS

We'll set the bounds for all the variables pi to [0, 1].

In [ ]:
n_var = 2

In [ ]:
variables = {}
for i in range(n_var):
    variables[f"p{i}"] = [0, 1]

vocs = VOCS(
    variables=variables,
    objectives={"f": "MINIMIZE"},
)

In [ ]:
vocs

#### Evaluator

In [ ]:
evaluator = Evaluator(function=f_test)

#### Generator

In [ ]:
generator = RCDSGenerator(vocs=vocs)

In [ ]:
generator.model_dump()

#### Run the optimization

In [ ]:
X = Xopt(vocs=vocs, evaluator=evaluator, generator=generator)

In [ ]:
X.evaluate_data({"p0": 0.5, "p1": 0.5})

In [ ]:
for i in tqdm(range(400)):
    X.step()

### Visualization

In [ ]:
X.data.plot(y="f")